First mount your google drive to this collab script, in order to read in the dataset an other things

In [1]:
#Mount Drive

Next, we will import the necessary modules which are:

- pandas
- numpy
- matplotlib (pyplot)

In [2]:
#Import the modules using the 'import' command

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Take the data files and process them into a pandas dataframe

In [7]:
#TODO: Add the other data frames, and then combine. Also add collumn names
columnNames = ['Age', 'Sex', 'Chest Pain Type', 'Resting Blood Pressure', 'Cholesterol', 'Fasting Blood Sugar', 'Resting ECG', 'Max Heart Rate', 'Exercise Induced Angina', 'ST Depression', ]
heartDisease_DataFrame = pd.read_csv('processed_hungarian.data')

heartDisease_DataFrame.head()

,28,1,2,130,132,0,2.1,185,0.1,0.2,?,?.1,?.2,0.3
0,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
1,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
2,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
3,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0
4,32,0,2,105,198,0,0,165,0,0.0,?,?,?,0
